# MATH 582 Final Project
Author: Jace Kline 2881618

### Project Description & Motivation

In this project, I aim to use health and fitness metrics scraped from my personal Garmin Connect account to analyze correlational relationships and perform dimensionality reduction, regression, classification, and clustering. The data is scraped from the time period of early June through late November and additionally exist across different time scales - daily and weekly. The metrics observed in the data sets include Calories burned, stress level, resting heart rate (BPM), steps, intensity minutes, and sleep hours. I chose this project because of my recent entrance into the world of endurance sports. I aim to leverage data science techniques learned in this class to derive useful and interesting results for myself as I continue on my endurance sports journey.


In [ ]:
# imports

# base
import pandas as pd
import numpy as np
import time

# plotting
import matplotlib.pyplot as mp
import seaborn as sb
from IPython.display import display

# dimensionality reduction, feature selection, data splitting
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector

# regression
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, ARDRegression

# classification
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# load the cleaned data sets
dataloc = "./data/cleaned/"

daily = pd.read_csv(dataloc + "daily.csv").set_index('Date')
weekly = pd.read_csv(dataloc + "weekly.csv").set_index('Week')

dataset_labels = ["Daily", "Weekly"]
datasets = [daily, weekly]

In [ ]:
def iterate_datasets(procedure):
    for df, lbl in zip(datasets, dataset_labels):
        procedure(df, lbl)

## Data Sets

In this section, we will describe and show the data sets used in the project. As discussed previously, the data comes from Garmin Connect and exists across daily and weekly time granularities. Hence, we utilize a daily and weekly data set. The data sets and their associated features are listed below:

* Daily metrics
  * Date
  * Intensity Minutes - the amount of time in a given day spent doing intense exercise
  * Steps - the number of steps taken in a given day
  * BPM - the average resting heart rate for a given day
  * Calories - the total number of calories burned in a given day
  * Stress Level - a Garmin score indicating the level of stress exhibited in a given day
  * Sleep Hrs - the number of hours slept for a given night
  * Sleep Hrs Prev - the number of hours slept the previous night


* Weekly metrics
  * Week - the date marking the first day of a seven day period
  * Intensity Minutes - the total intensity minutes in a given week
  * Steps - the average daily steps for a given week
  * BPM - the average resting heart rate over a given week
  * Calories - the total number of calories burned in a given week
  * Stress Level - The average daily stress level (Garmin score) over a given week
  * Sleep Hrs - The average number of hours slept per night over the course of a given week

In [ ]:
# display the structure of the data sets

display(daily)
display(weekly)

## Feature Correlation

In this section, we aim to show the correlation plots of features in our datasets that feature metrics broken down by day, week, and month. These different levels of granularity offer unique insights into the relationship of our different health metrics over the course of time. The results that we obtain in this section will help us narrow the scope and focus of our further analysis.

In [ ]:
def display_correlations(df, lbl):
    print(f"{lbl} Correlations:")
    corr = df.corr()
    display(corr)
    
    sb.heatmap(corr, cmap="YlGnBu", annot=True)
    mp.show()

# for lbl, df in zip(dataset_labels, datasets):
#     display_correlations(df, lbl)

iterate_datasets(display_correlations)

### Discussion

These correlation plots show us that the number of hours of sleep per night are negatively correlated with all other features. This is most pronounced on the weekly scale as opposed to the daily scale. Additionally, we see that stress level and resting heart rate (BPM) are significantly positively correlated in both the daily and weekly time scales. This indicates that both acute and prolonged stress associate with an increase in heart rate. Intuitively, we see that steps, intensity minutes, and Calories burned are all significantly positively correlated. The overlapping information stored within all these features present the possible success of dimensionality reduction on the data.

## Dimensionality Reduction (PCA & SVD)

As touched on in the feature correlation discussion, the shared information and high correlations present between features in the data present the possible success of dimensionality reduction using principal component analysis (PCA) and singular value decomposition (SVD) on the data. The dimension-reduced data sets can then be utilized in further models leveraging these data sets.

In [ ]:
# SVD Reducer function

# X: Data set with D columns (features), N rows (samples)
# M: the number of dimensions to reduce to where M <= D
def svd_reduce(X, M):
    U, singular_values, Vt = np.linalg.svd(X.T, full_matrices=False, compute_uv=True)
    Sigma = np.diag(singular_values)
    return (U[:,0:M] @ Sigma[0:M,0:M] @ Vt[0:M,:]).T

In [ ]:
# PCA Reducer function

# X: Data set with D columns (features), N rows (samples)
# M: the number of dimensions to reduce to where M <= D
# transforms the shape of data matrix back to original space
def pca_reduce(X, M):
    pca = PCA(n_components=M)
    return pca.inverse_transform(pca.fit_transform(X))

In [ ]:
def svd_vs_pca_err(X):
    N, D = X.shape
    results = []

    for M in range(1, D):
        svd_err = np.linalg.norm(X - svd_reduce(X, M))
        pca_err = np.linalg.norm(X - pca_reduce(X, M))
        results.append((M, svd_err, pca_err))

    return pd.DataFrame(results, columns=['Dimensions', 'SVD Error', 'PCA Error']).set_index('Dimensions')

def dim_reduction_results(df, lbl):
    print(f"SVD vs PCA on the {lbl} data set:")
    res = svd_vs_pca_err(df.to_numpy())
    display(res)
    res.plot(kind='bar', logy=True)
    mp.show()

In [ ]:
iterate_datasets(dim_reduction_results)

### Discussion

The results above indicate to us that dimensionality reduction on our target data sets is feasible and fairly accurate when using at least 3 dimensions. We utilized the Frobenius Norm function to measure the error the PCA and SVD methods. We see that the the PCA reduction resulted in more accurate information preservation than that of the SVD reduction. The success of PCA in this case can be explained by the relatively high redundancy, and therefore covariance, exhibited via the similarities in features such as Calories burned and intensity minutes.

## Regression

In this section, we aim to utilize regression to predict each feature using the other features. To achieve successful regression results, we must first prune our data set features to only utilize features that result in useful predictive outcomes for the given problem at hand. To achieve this, we will utilize the forward sequential feature selector offered by SciKit Learn to test different combinations of features on a given regression input model. We will choose the set of features that performs the highest for the given model. We will iterate over each of the data sets and perform regression against each of the features.

In [ ]:
# Split the input DataFrame (df) into X and y based on the target regression feature name (y_feature)
def split_dataset(df, y_feature):
    
    X_features = np.array(df.columns)
    X_features = X_features[X_features != y_feature]

    X = df[X_features]
    y = df[y_feature]
    return X, y

# Find the optimal features for the given target
# return the optimal features, the score of the model on the training data, and the score of the model on the testing data
def evaluate_model(X, y, model):
    
    feature_selector = SequentialFeatureSelector(model, k_features='best', n_jobs=-1)

    feature_selector.fit(X, y)
    optimal_features = list(feature_selector.k_feature_names_)

    X_train, X_test, y_train, y_test = train_test_split(X[optimal_features], y, random_state=57)

    model.fit(X_train, y_train)
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)

    return optimal_features, train_score, test_score

In [ ]:
# The SciKit Learn regression models to test
model_labels = ["LinearRegression", "Lasso", "Ridge", "ElasticNet", "ARDRegression"]
models = [LinearRegression(), Lasso(), Ridge(), ElasticNet(), ARDRegression()]

# output the results of running all of the above regression models on the input data set and target feature
def regression_tests(df, df_lbl, y_feature):
    results = []
    X, y = split_dataset(df, y_feature)

    for model, lbl in zip(models, model_labels):
        start = time.time()
        ftrs, train_score, test_score = evaluate_model(X, y, model)
        time_taken = time.time() - start
        results.append((lbl, ftrs, train_score, test_score, time_taken))

    res_table = pd.DataFrame(results, columns=['Model Type', 'Feature Inputs', 'Train Score', 'Test Score', 'Time Taken']).set_index('Model Type') 
    print("Feature Selector: Forward Sequential")
    print(f"Data Set: {df_lbl}")
    print(f"Prediction Feature: {y_feature}")
    display(res_table)
    res_table[['Train Score', 'Test Score']].plot(
        kind='bar',
        title=f"Results of predicting feature '{y_feature}' on the {df_lbl} data set"
    )
    mp.show()
    return res_table


In [ ]:
targets = ['Intensity Minutes', 'Calories', 'Stress Level', 'Sleep Hrs']
results = []

# Iterate over the data sets
for df, df_lbl in zip(datasets, dataset_labels):
    # For each data set, perform regression tests on each of the features
    for y_feature in targets:
        res = regression_tests(df, df_lbl, y_feature)
        results.append((f"{y_feature} ({df_lbl})", res))

In [ ]:
# compile and show the results for the models across all runs
# show distinct results for predictions on the training and testing data
test_lbls = [test_lbl for test_lbl, _ in results]
for feature in ['Train Score', 'Test Score', 'Time Taken']:
    results_arr = np.vstack([res[feature].to_numpy() for _, res in results])
    model_performances = pd.DataFrame(results_arr, columns=model_labels, index=test_lbls)
    avgs = model_performances.mean()
    best_performer = (min if feature == 'Time Taken' else max)(zip(list(avgs.index), list(avgs)), key=lambda t: t[1])[0]

    print(f"\n\nMeasurement: {feature}")
    display(model_performances)

    print(f"Model Average Performances ({feature})")
    display(avgs)
    print(f"Best Average Performer ({feature}): {best_performer}")

### Discussion

The above results show us that some of the features are much better suited to prediction via a regression model. Particularly, predicting the following metrics lead to respectable results: daily intensity minutes, daily Calories, and weekly stress level. The lack of accuracy in performing regression across the other features displays the fact that our data lacks necessary information required for us to make reasonable predictions. Data is simply a rough snapshot of a subset of important factors that could dictate the outcome of a phenomemon in the real world, and it is evident that predicting factors such as sleep hours, resting heart rate, and other features requires additional information not captured in our data.

Another interesting observation that arises from our results above is the fact that the number of optimal features in each of the regression models is nearly always a proper subset of the total set of features. This implies that some of the features or combinations of the features being utilized in the model may actually result in reduced accuracy through overfitting or addition of noise. This highlights the fact that more features are not always better when constructing a model.

Lastly, we show the comparison of regression predictions across the different models used. This analysis highlights the fact that the models all behaved similarly across the test runs. However, the LinearRegression and Ridge models behaved best overall.

## Classification

In this section, we will utilize binary classification to attempt to determine whether a feature value lies above or below a given threshold placed at the median of the target feature.

In [ ]:
def split_dataset_classification(df, y_feature):
    X, y = split_dataset(df, y_feature)
    threshold = y.median()
    y_classification = (y >= threshold).apply(lambda b: 1 if b else -1)
    return X, y, y_classification, threshold

In [ ]:
# The SciKit Learn classification models to test
svc_kernels = ['poly', 'rbf']
model_labels = [f'SVC (kernel = {kernel})' for kernel in svc_kernels] +\
    ["GaussianNB", "MultinomialNB", "SGDClassifier", "DecisionTreeClassifier", "RandomForestClassifier"]
models = [SVC(kernel=kernel) for kernel in svc_kernels] +\
    [GaussianNB(), MultinomialNB(), SGDClassifier(), DecisionTreeClassifier(), RandomForestClassifier()]

# output the results of running all of the above regression models on the input data set and target feature
def classification_tests(df, df_lbl, y_feature):
    results = []
    X, y, y_classification, threshold = split_dataset_classification(df, y_feature)

    for model, lbl in zip(models, model_labels):
        start = time.time()
        ftrs, train_score, test_score = evaluate_model(X, y_classification, model)
        time_taken = time.time() - start
        results.append((lbl, ftrs, train_score, test_score, time_taken))

    res_table = pd.DataFrame(results, columns=['Model Type', 'Feature Inputs', 'Train Score', 'Test Score', 'Time Taken']).set_index('Model Type') 
    print("Feature Selector: Forward Sequential")
    print(f"Data Set: {df_lbl}")
    print(f"Prediction Feature: {y_feature}")
    print(f"Classification Threshold: {threshold}")
    display(res_table)
    res_table[['Train Score', 'Test Score']].plot(
        kind='bar',
        title=f"Results of classifying feature '{y_feature}' on the {df_lbl} data set above or below threshold {threshold}"
    )
    mp.show()
    return res_table

In [ ]:
targets = ['Intensity Minutes', 'Calories', 'Stress Level', 'Sleep Hrs']
results = []

# Iterate over the data sets
for df, df_lbl in zip(datasets, dataset_labels):
    # For each data set, perform regression tests on each of the features
    for y_feature in targets:
        res = classification_tests(df, df_lbl, y_feature)
        results.append((f"{y_feature} ({df_lbl})", res))

In [ ]:
# compile and show the results for the models across all runs
# show distinct results for predictions on the training and testing data
test_lbls = [test_lbl for test_lbl, _ in results]
for feature in ['Train Score', 'Test Score', 'Time Taken']:
    results_arr = np.vstack([res[feature].to_numpy() for _, res in results])
    model_performances = pd.DataFrame(results_arr, columns=model_labels, index=test_lbls)
    avgs = model_performances.mean()
    best_performer = (min if feature == 'Time Taken' else max)(zip(list(avgs.index), list(avgs)), key=lambda t: t[1])[0]

    print(f"\n\nMeasurement: {feature}")
    display(model_performances)

    print(f"Model Average Performances ({feature})")
    display(avgs)
    print(f"Best Average Performer ({feature}): {best_performer}")